In [1]:
import pandas as pd
import numpy as np

import warnings
import pandas_profiling

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import IsolationForest

# Show all columns
pd.set_option('display.max_columns', None)

# Ignore all warnings
warnings.filterwarnings("ignore")

#!pip install scikit-learn~=1.1
#import sklearn
#print(sklearn.__version__)

## 1. Data Quick Look

In [2]:
df_train = pd.read_csv('../data/airline_train.csv')

In [3]:
df_train 

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,103899,94171,Female,disloyal Customer,23,Business travel,Eco,192,2,1,2,3,2,2,2,2,3,1,4,2,3,2,3,0.0,neutral or dissatisfied
103900,103900,73097,Male,Loyal Customer,49,Business travel,Business,2347,4,4,4,4,2,4,5,5,5,5,5,5,5,4,0,0.0,satisfied
103901,103901,68825,Male,disloyal Customer,30,Business travel,Business,1995,1,1,1,3,4,1,5,4,3,2,4,5,5,4,7,14.0,neutral or dissatisfied
103902,103902,54173,Female,disloyal Customer,22,Business travel,Eco,1000,1,1,1,5,1,1,1,1,4,5,1,5,4,1,0,0.0,neutral or dissatisfied


In [4]:
'''
profile = df_train.profile_report(
    title="Report without correlations",
    correlations=None,
)
profile.to_file("../data/airline_train_profile.html")
'''

'\nprofile = df_train.profile_report(\n    title="Report without correlations",\n    correlations=None,\n)\nprofile.to_file("../data/airline_train_profile.html")\n'

In [5]:
# Data Type and Null Values
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [6]:
id_cols = ['Unnamed: 0', 'id']
satisfaction_cols = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 
                    'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 
                    'Inflight entertainment', 'On-board service', 'Leg room service', 
                    'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness',
                    'satisfaction']
outlier_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']
target_col = ['satisfaction']
null_cols = df_train.loc[:, df_train.isna().any()].columns.tolist() #Arrival Delay in Minutes

## 2. Preprocessing
### 2.1. Imputing NA values

In [7]:
null_rows_idx = df_train.isnull().any(axis=1)
df_train.loc[null_rows_idx].head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
213,213,49608,Female,Loyal Customer,38,Business travel,Eco,109,5,3,3,3,5,5,5,5,5,2,4,1,1,5,31,NaN,satisfied
1124,1124,73442,Male,Loyal Customer,53,Personal Travel,Eco,1012,3,2,3,4,4,3,4,4,4,4,4,3,3,4,38,NaN,neutral or dissatisfied
1529,1529,71178,Male,Loyal Customer,39,Business travel,Business,733,2,5,5,5,2,4,3,2,2,2,2,2,2,3,11,NaN,neutral or dissatisfied
2004,2004,72940,Female,disloyal Customer,26,Business travel,Business,1035,3,3,3,1,2,3,2,2,3,3,4,5,5,2,41,NaN,neutral or dissatisfied
2108,2108,116374,Female,Loyal Customer,24,Personal Travel,Eco,417,2,1,2,2,5,2,5,5,1,4,2,1,2,5,1,NaN,neutral or dissatisfied


In [8]:
imputer = SimpleImputer(strategy="median")
df_train_null = df_train[null_cols]
imputer.fit(df_train_null)
#imputer.strategy is 'median'
X_nonull = imputer.transform(df_train_null)
df_imputed = pd.DataFrame(X_nonull, columns=df_train_null.columns,
                          index=df_train_null.index)

In [9]:
df_train_cleaned = df_train.copy()
df_train_cleaned[null_cols] = X_nonull
df_train_cleaned.loc[null_rows_idx].head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
213,213,49608,Female,Loyal Customer,38,Business travel,Eco,109,5,3,3,3,5,5,5,5,5,2,4,1,1,5,31,0.0,satisfied
1124,1124,73442,Male,Loyal Customer,53,Personal Travel,Eco,1012,3,2,3,4,4,3,4,4,4,4,4,3,3,4,38,0.0,neutral or dissatisfied
1529,1529,71178,Male,Loyal Customer,39,Business travel,Business,733,2,5,5,5,2,4,3,2,2,2,2,2,2,3,11,0.0,neutral or dissatisfied
2004,2004,72940,Female,disloyal Customer,26,Business travel,Business,1035,3,3,3,1,2,3,2,2,3,3,4,5,5,2,41,0.0,neutral or dissatisfied
2108,2108,116374,Female,Loyal Customer,24,Personal Travel,Eco,417,2,1,2,2,5,2,5,5,1,4,2,1,2,5,1,0.0,neutral or dissatisfied


### 2.2. Handling Outliers

In [10]:
X = df_train_cleaned[outlier_cols]

isolation_forest = IsolationForest(random_state=42, contamination = 0.01)
outlier_pred = isolation_forest.fit_predict(X)

df_train_no_outlier = df_train_cleaned.iloc[outlier_pred == 1]

df_train_outlier = df_train_cleaned.iloc[outlier_pred == -1]
print('dropped rows:', df_train_outlier.shape[0])

dropped rows: 1040


In [11]:
df_train_outlier[outlier_cols].describe()

,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes
count,1040.000000,1040.000000,1040.000000,1040.000000
mean,39.375962,1556.817308,259.946154,263.417308
std,16.380051,1195.112287,119.122526,118.711615
min,7.000000,67.000000,69.000000,81.000000
25%,25.000000,502.250000,194.000000,193.000000
50%,40.000000,1099.000000,232.000000,238.000000
75%,52.000000,2475.000000,295.000000,298.250000
max,80.000000,4963.000000,1592.000000,1584.000000


### 2.3. Normalization Pipeline

In [68]:
log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, inverse_func=np.exp, feature_names_out="one-to-one"),
    StandardScaler())

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))

default_num_pipeline = make_pipeline(StandardScaler())

#("cat", cat_pipeline, make_column_selector(dtype_include=object)),
preprocessing = ColumnTransformer([
    #("log", log_pipeline, ['Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']),
    ("cat", cat_pipeline, categorical_cols),
    ("std", default_num_pipeline, ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']),
    ],
    remainder='passthrough')  # one column remaining: housing_median_age

In [77]:
airplane = df_train_no_outlier.copy()
airplane['Arrival Delay in Minutes'] = airplane['Arrival Delay in Minutes'].astype(int)
airplane.drop(columns = ['Unnamed: 0', 'id'], inplace = True)
airplane

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,Female,disloyal Customer,23,Business travel,Eco,192,2,1,2,3,2,2,2,2,3,1,4,2,3,2,3,0,neutral or dissatisfied
103900,Male,Loyal Customer,49,Business travel,Business,2347,4,4,4,4,2,4,5,5,5,5,5,5,5,4,0,0,satisfied
103901,Male,disloyal Customer,30,Business travel,Business,1995,1,1,1,3,4,1,5,4,3,2,4,5,5,4,7,14,neutral or dissatisfied
103902,Female,disloyal Customer,22,Business travel,Eco,1000,1,1,1,5,1,1,1,1,4,5,1,5,4,1,0,0,neutral or dissatisfied


In [78]:
airplane_normalized = preprocessing.fit_transform(airplane)
airplane_normalized = pd.DataFrame(airplane_normalized,
    columns=preprocessing.get_feature_names_out(),
    index=airplane.index)
airplane_normalized.head(2)

,cat__Gender_Female,cat__Gender_Male,cat__Customer Type_Loyal Customer,cat__Customer Type_disloyal Customer,cat__Type of Travel_Business travel,cat__Type of Travel_Personal Travel,cat__Class_Business,cat__Class_Eco,cat__Class_Eco Plus,std__Age,std__Flight Distance,std__Departure Delay in Minutes,std__Arrival Delay in Minutes,remainder__Inflight wifi service,remainder__Departure/Arrival time convenient,remainder__Ease of Online booking,remainder__Gate location,remainder__Food and drink,remainder__Online boarding,remainder__Seat comfort,remainder__Inflight entertainment,remainder__On-board service,remainder__Leg room service,remainder__Baggage handling,remainder__Checkin service,remainder__Inflight service,remainder__Cleanliness,remainder__satisfaction
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.746813,-0.729928,0.472121,0.198061,3,4,3,1,5,3,5,5,4,3,4,4,5,5,neutral or dissatisfied
1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,-0.952197,-0.956229,-0.4227,-0.243968,3,2,3,3,1,3,1,1,1,5,3,1,4,1,neutral or dissatisfied


In [73]:
airplane_normalized.shape[1]

27

### 2.4. Save processed file to local

In [79]:
airplane.to_csv('../data/airplane_train_processed.csv', index=False)
airplane_normalized.to_csv('../data/airplane_train_processed_normalized.csv', index=False)